In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/2024-1 data-mining/kaggle_second'

/content/drive/MyDrive/2024-1 data-mining/kaggle_second


In [2]:
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 MB 44.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.4 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
import time
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Define parameters
batch_size = 32
num_epochs = 25
learning_rate = 0.001
num_classes = 6

# Define transform
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(320),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

# Load train dataset
data_dir = 'data'
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train'])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Custom dataset for test data
class TestDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.img_paths = [os.path.join(img_dir, img) for img in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, img))]

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, os.path.basename(img_path)

test_dataset = TestDataset(os.path.join(data_dir, 'test'), data_transforms['test'])
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load pre-trained InceptionV3 model and modify it for our task
model = models.inception_v3(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train and evaluate the model
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            data_loader = train_loader if phase == 'train' else test_loader
            for inputs, labels in data_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)

                    if isinstance(outputs, tuple):
                        outputs = outputs[0]  # Get the logits from InceptionOutputs

                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(data_loader.dataset)
            epoch_acc = running_corrects.double() / len(data_loader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'train' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best train Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

# Train the model
model = train_model(model, criterion, optimizer, num_epochs=num_epochs)

# Save the model
torch.save(model.state_dict(), 'inception_v3_model.pth')

# Generate predictions for test dataset and save to CSV
model.eval()
predictions = []

with torch.no_grad():
    for inputs, img_names in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)

        if isinstance(outputs, tuple):
            outputs = outputs[0]  # Get the logits from InceptionOutputs

        _, preds = torch.max(outputs, 1)
        for img_name, pred in zip(img_names, preds):
            predictions.append({'Id': img_name, 'Category': pred.item()})

df = pd.DataFrame(predictions)
df.to_csv('test_predictions.csv', index=False)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/24
----------
train Loss: 1.0533 Acc: 0.6165

Epoch 1/24
----------
train Loss: 0.8578 Acc: 0.6990

Epoch 2/24
----------
train Loss: 0.7068 Acc: 0.7406

Epoch 3/24
----------
train Loss: 0.8037 Acc: 0.7102

Epoch 4/24
----------
train Loss: 0.7289 Acc: 0.7398

Epoch 5/24
----------
train Loss: 0.7282 Acc: 0.7446

Epoch 6/24
----------
train Loss: 0.5739 Acc: 0.7890

Epoch 7/24
----------
train Loss: 0.6155 Acc: 0.7830

Epoch 8/24
----------
train Loss: 0.6973 Acc: 0.7494

Epoch 9/24
----------
train Loss: 0.5592 Acc: 0.7958

Epoch 10/24
----------
train Loss: 0.5707 Acc: 0.8006

Epoch 11/24
----------
train Loss: 0.5345 Acc: 0.8090

Epoch 12/24
----------
train Loss: 0.4871 Acc: 0.8227

Epoch 13/24
----------
train Loss: 0.5082 Acc: 0.8179

Epoch 14/24
----------
train Loss: 0.5496 Acc: 0.7986

Epoch 15/24
----------
train Loss: 0.5392 Acc: 0.8078

Epoch 16/24
----------
train Loss: 0.4412 Acc: 0.8407

Epoch 17/24
----------
train Loss: 0.4526 Acc: 0.8339

Epoch 18/24
--------

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
import time
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Define parameters
batch_size = 32
num_epochs = 50
learning_rate = 0.0001
num_classes = 6

# Define transform
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(320),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

# Load train dataset
data_dir = 'data'
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train'])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Custom dataset for test data
class TestDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.img_paths = [os.path.join(img_dir, img) for img in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, img))]

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, os.path.basename(img_path)

test_dataset = TestDataset(os.path.join(data_dir, 'test'), data_transforms['test'])
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load pre-trained InceptionV3 model and modify it for our task
model = models.inception_v3(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def train_model(model, criterion, optimizer, num_epochs=50, early_stopping_patience=5):
    since = time.time()
    best_model_wts = model.state_dict()
    best_acc = 0.0
    epochs_no_improve = 0  # Counter for epochs with no improvement

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            data_loader = train_loader if phase == 'train' else test_loader
            for inputs, labels in data_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)

                    if isinstance(outputs, tuple):
                        outputs = outputs[0]  # Get the logits from InceptionOutputs

                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(data_loader.dataset)
            epoch_acc = running_corrects.double() / len(data_loader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'train' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

        print()

        if epochs_no_improve >= early_stopping_patience:
            print(f'Early stopping triggered after {epoch + 1} epochs without improvement.')
            break

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best train Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

# Train the model with early stopping
model = train_model(model, criterion, optimizer, num_epochs=num_epochs, early_stopping_patience=5)

# Save the model
torch.save(model.state_dict(), 'inception_v3_model.pth')

# Generate predictions for test dataset and save to CSV
model.eval()
predictions = []

with torch.no_grad():
    for inputs, img_names in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)

        if isinstance(outputs, tuple):
            outputs = outputs[0]  # Get the logits from InceptionOutputs

        _, preds = torch.max(outputs, 1)
        for img_name, pred in zip(img_names, preds):
            predictions.append({'Id': img_name, 'Category': pred.item()})

# Sort predictions by image filenames converted to integers
predictions.sort(key=lambda x: int(os.path.splitext(x['Id'])[0]))

df = pd.DataFrame(predictions)
df.to_csv('test_predictions_2.csv', index=False)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 169MB/s] 


Epoch 0/49
----------
train Loss: 0.9308 Acc: 0.6797

Epoch 1/49
----------
train Loss: 0.4089 Acc: 0.8591

Epoch 2/49
----------
train Loss: 0.3308 Acc: 0.8927

Epoch 3/49
----------
train Loss: 0.2826 Acc: 0.9023

Epoch 4/49
----------
train Loss: 0.2954 Acc: 0.8971

Epoch 5/49
----------
train Loss: 0.2602 Acc: 0.9071

Epoch 6/49
----------
train Loss: 0.2441 Acc: 0.9131

Epoch 7/49
----------
train Loss: 0.2253 Acc: 0.9195

Epoch 8/49
----------
train Loss: 0.2424 Acc: 0.9159

Epoch 9/49
----------
train Loss: 0.2160 Acc: 0.9215

Epoch 10/49
----------
train Loss: 0.2439 Acc: 0.9187

Epoch 11/49
----------
train Loss: 0.2316 Acc: 0.9179

Epoch 12/49
----------
train Loss: 0.2238 Acc: 0.9167

Epoch 13/49
----------
train Loss: 0.2212 Acc: 0.9207

Epoch 14/49
----------
train Loss: 0.2127 Acc: 0.9235

Epoch 15/49
----------
train Loss: 0.2183 Acc: 0.9251

Epoch 16/49
----------
train Loss: 0.2047 Acc: 0.9259

Epoch 17/49
----------
train Loss: 0.1733 Acc: 0.9384

Epoch 18/49
--------

In [3]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 961.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.3/410.3 kB 41.2 MB/s eta 0:00:00


In [7]:
cd '/Users/yubeen/Desktop/for-student'

/Users/yubeen/Desktop/for-student


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
import time
import os
from PIL import ImageFile
from timm import create_model
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Define parameters
batch_size = 128
num_epochs = 50
learning_rate = 0.0001
num_classes = 6

# Define transform
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

# Load train dataset
data_dir = 'data'
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train'])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Custom dataset for test data
class TestDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.img_paths = [os.path.join(img_dir, img) for img in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, img))]

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, os.path.basename(img_path)

test_dataset = TestDataset(os.path.join(data_dir, 'test'), data_transforms['test'])
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load pre-trained EfficientNet model and modify it for our task
model = create_model('tf_efficientnet_b0', pretrained=True)
num_features = model.classifier.in_features
model.classifier = nn.Linear(num_features, num_classes)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def train_model(model, criterion, optimizer, num_epochs=num_epochs, early_stopping_patience=5):
    since = time.time()
    best_model_wts = model.state_dict()
    best_acc = 0.0
    epochs_no_improve = 0  # Counter for epochs with no improvement

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            data_loader = train_loader if phase == 'train' else test_loader
            for inputs, labels in data_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)

                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(data_loader.dataset)
            epoch_acc = running_corrects.double() / len(data_loader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'train' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

        print()

        if epochs_no_improve >= early_stopping_patience:
            print(f'Early stopping triggered after {epoch + 1} epochs without improvement.')
            break

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best train Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

# Train the model with early stopping
model = train_model(model, criterion, optimizer, num_epochs=num_epochs, early_stopping_patience=5)

# Save the model
torch.save(model.state_dict(), 'efficientnet_model.pth')

# Generate predictions for test dataset and save to CSV
model.eval()
predictions = []

with torch.no_grad():
    for inputs, img_names in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)

        _, preds = torch.max(outputs, 1)
        for img_name, pred in zip(img_names, preds):
            predictions.append({'Id': img_name, 'Category': pred.item()})

# Sort predictions by image filenames converted to integers
predictions.sort(key=lambda x: int(os.path.splitext(x['Id'])[0]))

df = pd.DataFrame(predictions)
df.to_csv('test_predictions_efficientnet_128_0.0001.csv', index=False)

model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

Epoch 0/49
----------


OSError: Caught OSError in DataLoader worker process 2.
Original Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/torchvision/datasets/folder.py", line 245, in __getitem__
    sample = self.loader(path)
             ^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/torchvision/datasets/f

In [5]:
!pwd

/Users/yubeen/Desktop
